### Install the required libraries

In [ ]:
!pip install pandas fastparquet plotly

### Read Data

In [1]:
import pandas as pd

df = pd.read_parquet("data/odeal_hackathon.parquet", engine='fastparquet')

### Function to calculate customer churn and transaction frequency from a DataFrame based on transaction history and a specified inactivity threshold

In [2]:
def analyze_customer_churn(df, id_col='id', transaction_date_col='IslemTarih',
                           transaction_id_col='IslemID', churn_days_threshold=90):
    """
    Analyze customer churn from transaction data.

    Parameters
    ----------
    df : pandas.DataFrame
        A DataFrame containing the transaction data.
    id_col : str, optional
        The name of the column that contains the customer ID. Default is 'CustomerID'.
    transaction_date_col : str, optional
        The name of the column that contains the transaction date. Default is 'TransactionDate'.
    transaction_id_col : str, optional
        The name of the column that contains the transaction ID. Default is 'TransactionID'.
    churn_days_threshold : int, optional
        The number of days without transactions to consider a customer as churned. Default is 90.

    Returns
    -------
    pandas.DataFrame
        A DataFrame with churn analysis by customer, including churn status and transaction frequency.

    Examples
    --------
    >>> df = pd.DataFrame({
    ...     'CustomerID': [1, 1, 2, 2, 3],
    ...     'TransactionDate': ['2021-01-01', '2021-01-02', '2021-01-02', '2021-02-01', '2021-03-01'],
    ...     'TransactionID': [101, 102, 103, 104, 105]
    ... })
    >>> result = analyze_customer_churn(df)
    >>> result
    """
    df[transaction_date_col] = pd.to_datetime(df[transaction_date_col])

    transaction_dates = df.groupby(id_col)[transaction_date_col].agg(['min', 'max'])
    transaction_dates.columns = ['FirstTransactionDate', 'LastTransactionDate']

    latest_date = df[transaction_date_col].max()
    days_since_last_transaction = (latest_date - transaction_dates['LastTransactionDate']).dt.days

    churn_status = days_since_last_transaction > churn_days_threshold

    df['Month'] = df[transaction_date_col].dt.to_period('M')
    transaction_freq = df.groupby([id_col, 'Month'])[transaction_id_col].count().groupby(level=0).mean()

    churn_analysis = pd.concat([
        transaction_dates,
        days_since_last_transaction.rename('DaysSinceLastTransaction'),
        churn_status.rename('Churn'),
        transaction_freq.rename('TransactionFrequency')
    ], axis=1).reset_index()

    return churn_analysis

### Show Result:

In [3]:
result = analyze_customer_churn(df)

result

,id,FirstTransactionDate,LastTransactionDate,DaysSinceLastTransaction,Churn,TransactionFrequency
0,301002470,2015-09-18 14:35:04,2023-10-26 10:47:00,0,False,442.714286
1,301002583,2014-09-25 10:08:52,2020-12-23 12:28:28,1037,True,9.533333
2,301003354,2022-11-21 18:05:00,2023-10-26 16:59:00,0,False,2551.333333
3,301006906,2016-01-29 11:00:43,2022-09-12 18:30:00,408,True,618.350649
4,301009366,2019-06-27 17:31:14,2022-09-19 18:01:00,401,True,478.225000
5,301009412,2016-08-26 11:00:35,2023-10-03 17:56:28,22,False,1.333333
6,301009682,2016-05-19 10:29:01,2023-10-26 16:39:00,0,False,176.011765
7,301010389,2016-06-15 15:12:25,2020-05-10 11:09:14,1264,True,510.424242
8,301012873,2018-12-07 12:52:00,2023-10-26 15:06:00,0,False,616.800000
9,301014561,2017-08-10 14:22:03,2023-10-26 16:55:00,0,False,1061.166667


## Sütun Açıklamaları:

- `id`:
  - **Açıklama**: Bu numerik ID, veri setindeki her müşteriyi eşsiz bir şekilde temsil eder. Her ID sadece bir müşteriyle ilişkilendirilmiştir.

- `FirstTransactionDate`:
  - **Açıklama**: Müşterinin yaptığı ilk işlemin tarihi ve saati.

- `LastTransactionDate`:
  - **Açıklama**: Verinin kaydedildiği tarihten önce müşteri tarafından yapılan son işlemin tarihi ve saati.

- `DaysSinceLastTransaction`:
  - **Açıklama**: Müşterinin son işleminden bu yana geçen gün sayısı.

- `Churn`:
  - **Açıklama**: Müşterinin belirli bir süre içinde (90 gün) herhangi bir işlem yapmaması durumunda müşteri kaybı yaşanıp yaşanmadığını belirten bir gösterge. (Bu, müşterinin son işlem tarihinden itibaren belirli bir süre içinde (90 gün) herhangi bir işlem yapmaması durumunda `True` olarak işaretlenir. Eğer müşteri bu süre içinde aktifse `False` olarak belirtilir.) 

- `TransactionFrequency`:
  - **Açıklama**: Müşterinin ortalama işlem sıklığı.

Her bir sütunun veri seti içindeki kullanımı ve analizi, müşteri davranışlarının daha iyi anlaşılmasına ve iş stratejilerinin geliştirilmesine katkı sağlayacaktır.

### Calculating Customer Churn

In [4]:
result.Churn.value_counts() * 100 / len(result)

Churn
False    58.974359
True     41.025641
Name: count, dtype: float64

### Visualize the churn rate 

In [5]:
import pandas as pd
import plotly.express as px

def create_churn_rate_pie_chart(dataframe, churn_column='Churn', title='<b>Churn Rate</b>', show_fig=True):
    """
    Creates a pie chart visualizing churn rates using Plotly.

    Parameters
    ----------
    dataframe : DataFrame
        A pandas DataFrame containing the data.
    churn_column : str, optional
        The column name in dataframe that contains the churn statuses, by default 'Churn'.
    title : str, optional
        Title of the pie chart, by default '<b>Churn Rate</b>'.
    show_fig : bool, optional
        If True, the figure will be displayed in the browser, by default True.

    Returns
    -------
    figure : plotly.graph_objs._figure.Figure
        A plotly Figure object representing the pie chart.

    Examples
    --------
    >>> create_churn_rate_pie_chart(result_df, churn_column='Churn')
    This will create and display a pie chart with churn data taken from `result_df`.

    """
    dataframe['Churn_Label'] = dataframe[churn_column].replace({True: 'Churned', False: 'Retained'})
    
    churn_counts = dataframe.groupby('Churn_Label').size().reset_index(name='Count')
    
    churn_counts['Percentage'] = (churn_counts['Count'] / churn_counts['Count'].sum() * 100).apply(lambda x: f"{x:.2f}%")

    fig = px.pie(
        churn_counts,
        names='Churn_Label',
        values='Count',
        title=title,
        color_discrete_sequence=px.colors.sequential.Agsunset,
        hole=0.3
    )

    fig.update_layout(
        title_font_size=24,
        title_font_color="navy",
        title_font_family="Arial",
        title_x=0.5,
        legend_title_text='Status'
    )

    fig.update_traces(
        textinfo='percent+label',
        texttemplate='%{label}: %{customdata[0]}',
        hovertemplate='%{label}: %{percent:.2f}%',
        pull=[0.1 if label == 'Churned' else 0 for label in churn_counts['Churn_Label']],
        marker=dict(line=dict(color='#000000', width=2)),
        customdata=churn_counts[['Percentage']]
    )

    if show_fig:
        fig.show(renderer='browser')

    return fig

In [6]:
create_churn_rate_pie_chart(result)